# Depression Detection Model for Twitter Accounts
Santiago Paiz

Ekaterina Podruzhko

Abdullah Al Saidi

Hisham Al Hashmi



# Discovery

In [2]:
import pandas as pd
import os

In [3]:
DATA_DEPRESSION_PATH = './data/depression/'

In [4]:
depression_dfs = {
    f.strip('.csv').replace(' ', "_"): pd.read_csv(DATA_DEPRESSION_PATH + f) for f in os.listdir(DATA_DEPRESSION_PATH)
    }

In [9]:
for key, value in depression_dfs.items():
    print(f'TITLE: {key}')

TITLE: I_suffer_from_depression
TITLE: users_timeline
TITLE: I_am_fighting_depression
TITLE: I_am_diagnosed_with_depression


In [10]:
depression_dfs['I_suffer_from_depression'].sample(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
72,1434523539474898952,1434523539474898952,2021-09-05 14:27:18 UTC,2021-09-05,14:27:18,0,4183990572,pinkiscupcake22,Pinkie_Pie,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
448,1432580309531316224,1432523141314646020,2021-08-31 05:45:36 UTC,2021-08-31,05:45:36,0,42230777,ddeshopper,Doreen,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'JustAMomNamedMP', 'name': 'M...",NaN,NaN,NaN,NaN


In [5]:
depression_dfs['I_am_fighting_depression'].sample(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
16,1433548644800622610,1433548644800622610,2021-09-02 21:53:25 UTC,2021-09-02,21:53:25,0,1375720916068532224,jayesstoo,JS2,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
68,1429419914650177537,1429419914650177537,2021-08-22 12:27:19 UTC,2021-08-22,12:27:19,0,708110531027140613,iambilah,Bilah 💎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
depression_dfs['users_timeline'].sample(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
36603,1433667189899505677,1433666161749766181,2021-09-03 05:44:28 UTC,2021-09-03,05:44:28,0,35043239,melkoalabrown07,MelBrown💔MOM RIP6/28/21 SoarHigh😭GH Actors JaSam,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
16993,1432981247207301120,1432947676790902787,2021-09-01 08:18:47 UTC,2021-09-01,08:18:47,0,2487878923,prettyfufu,Grumpy Bee 🐝| got her shots 💉,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'caridiva', 'name': 'Cari Gol...",NaN,NaN,NaN,NaN


In [44]:
for key, value in depression_dfs.items():
    print(f'TITLE: {key}')
    print()

TITLE: I_suffer_from_depression
Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

TITLE: users_timeline
Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       't

In [37]:
import collections

In [45]:
if(collections.Counter(depression_dfs['I_suffer_from_depression'])==collections.Counter(depression_dfs['I_am_fighting_depression']) \
    and collections.Counter(depression_dfs['I_am_fighting_depression']) == collections.Counter(depression_dfs['I_am_diagnosed_with_depression'])
    and collections.Counter(depression_dfs['I_am_fighting_depression']) == collections.Counter(depression_dfs['users_timeline'])):
    print("Equal")
else:
    print("Non equal")


Equal


There are 4 different data tables, however all of them have the same schema, in order to have them all in a single data frame we join them together adding a new column which represented in which data it was before the merger.

In order to make it easier we add one column to each of the data frame with the names of which the users is. 

In [46]:
for key, value in depression_dfs.items():
    value['depr_level'] = key


In [51]:
depression_dfs['I_suffer_from_depression'].sample(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,depr_level
443,1432658443429502977,1432658443429502977,2021-08-31 10:56:05 UTC,2021-08-31,10:56:05,0,1417122361745154049,crldwnbgt_,Dawn,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_suffer_from_depression
343,1433208710982033411,1433117975310123019,2021-09-01 23:22:39 UTC,2021-09-01,23:22:39,0,2789574570,parsleyjane8,Jane Parsley 🆘,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'clairermassey', 'name': '𝓒𝓵𝓪...",NaN,NaN,NaN,NaN,I_suffer_from_depression


In [55]:
# here is the concat of every df
derp_data = pd.concat(list(depression_dfs.values()))

In [58]:
derp_data.sample(5)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,depr_level
4961,1431668815885131780,1431668815885131780,2021-08-28 17:23:39 UTC,2021-08-28,17:23:39,0,1426671062356533248,matthasadhd,matt,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
35192,1433478420881498112,1433478420881498112,2021-09-02 17:14:22 UTC,2021-09-02,17:14:22,0,1390740031845789697,danieltuesta7,🇨🇺Daniel 🇺🇸Tuesta 🇵🇪,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
23773,1434167837141131268,1433892527262863370,2021-09-04 14:53:52 UTC,2021-09-04,14:53:52,0,3084322089,papa49tw,Papa Steve,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'LoveMyKiaraM', 'name': 'Raye...",NaN,NaN,NaN,NaN,users_timeline
42425,1433143287150485513,1433120010633887748,2021-09-01 19:02:40 UTC,2021-09-01,19:02:40,0,181659599,beacher61,Kathleen Morgan,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'MarieanneDH', 'name': 'RedBe...",NaN,NaN,NaN,NaN,users_timeline
7856,1435457240194904064,1435457240194904064,2021-09-08 04:17:30 UTC,2021-09-08,04:17:30,0,1174882312082714626,_deshanna_,-Nex/Deshanna-¹⁴³,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline


# Data Analysis

# Something